In [1]:
import json
import random
from pathlib import Path

import pandas as pd

%load_ext autoreload
%autoreload 2
Path.ls = lambda x: list(x.iterdir())

In [2]:
filepath = Path("NHP 2018-pages-266-277.pdf").resolve()
import json

In [3]:
# !pip uninstall -y camelot
# !pip install camelot-py
# !pip install camelot-py[cv]
# !pip install camelot-py[all]
# !pip install pdfplumber
# import camelot

In [4]:
# tables = camelot.read_pdf("NHP 2018-pages-266-277.pdf", flavour="stream", compress=True)

In [5]:
import pdfplumber

pagewise_df = []
with pdfplumber.open(filepath) as pdf:
    for page in pdf.pages[:-1]:
        page_df = pd.DataFrame(
            page.extract_tables(
                {
                    "vertical_strategy": "text",
                    "horizontal_strategy": "text",
                    "keep_blank_chars": True,
#                     "min_words_horizontal": 6,
#                     "text_tolerance":15
                }
            )[0][4:],
            #             columns=["S.No", "State/UT", "MedicalCollegeName", "City/Town", "Govt/Private", "AdmissionCapacity", "BedsInAttachedHospital"]
        )

        pagewise_df.append(page_df)

In [6]:
df = pagewise_df[0]

In [7]:
def process_df(df):
#     print(df.head())
    if len(df.columns) == 7:
        df.columns = [
            "S.No",
            "State/UT",
            "MedicalCollegeName",
            "City/Town",
            "Govt/Private",
            "AdmissionCapacity",
            "BedsInAttachedHospital",
        ]
        return df
    elif len(df.columns) == 5:
        df.columns = [
            "S.No",
            "MedicalCollegeName",
            "City/Town || Govt/Private",
            "AdmissionCapacity",
            "BedsInAttachedHospital",
        ]
    else:
        df.columns = [
            "S.No",
            "State/UT",
            "MedicalCollegeName",
            "City/Town || Govt/Private",
            "AdmissionCapacity",
            "BedsInAttachedHospital",
        ]

    def split_columns(field):
        entries = field.split()
        try:
            insti_status = entries[-1]
            city = " ".join(entries[:-1])
        except IndexError:
            insti_status, city = None, None
        return [city, insti_status]

    try:
        split_df = df["City/Town || Govt/Private"].apply(split_columns).tolist()
    except AttributeError:
        df.columns = [
            "S.No",
            #             "State/UT",
            "MedicalCollegeName",
            "City/Town",
            "Govt/Private",
            "AdmissionCapacity",
            "BedsInAttachedHospital",
        ]
        return df
    #         split_df = df["City/Town || Govt/Private"].apply(split_columns).tolist()
    split_df = pd.DataFrame(split_df, columns=["City/Town", "Govt/Private"])
    df = pd.merge(df, split_df, right_index=True, left_index=True)
    df.drop(columns=["City/Town || Govt/Private"], inplace=True)
    return df


process_df(pagewise_df[9])

,S.No,State/UT,MedicalCollegeName,City/Town,Govt/Private,AdmissionCapacity,BedsInAttachedHospital
0,393,None,"Mallareddy\tMedical\tCollege\tfor\tWomens,\tHy...",Hyderabad,Society,0,150
1,394,None,"Mamta\tMedical\tCollege,\tKhammam",Khammam,Trust,150,NA
2,395,None,"Mediciti\tInstitute\tOf\tMedical\tSciences,\tG...",Ghanpur,Trust,100,570
3,396,None,"MNR\tMedical\tCollege\t&\tHospital,\tSangareddy",Sangareddy,Trust,100,570
4,397,None,"Osmania\tMedical\tCollege,\tHyderabad",Hyderabad,Govt.,250,1168
5,398,None,"Prathima\tInstitute\tOf\tMedical\tSciences,\tK...",Karimnagar,Trust,200,780
6,399,None,R.V.M.\tInstitute\tof\tMedical\tSciences\tand\...,Medak,Trust,0,NA
7,400,None,Rajiv\tGandhi\tInstitute\tof\tMedical\tScience...,Adilabad,Govt.,100,750
8,401,None,"S\tV\tS\tMedical\tCollege,\tMehboobnagar",Mehboobnagar,Trust,100,750
9,402,None,"Shadan\tInstitute\tof\tMedical\tSciences,Resea...",Peerancheru,Society,150,750


In [9]:
for i in range(len(pagewise_df[:-1])):
    temp_df = process_df(pagewise_df[i])
    temp_df.to_csv(f"Page{i}_IsolationGovtBeds.csv", index=False)